# *Tesouro Prefixado* **with half-yearly interests (NTN-F)** 

$$V_{P} = \frac{cp}{(1 + \frac{i}{100})^{\frac{du_1}{252}}} + \frac{cp}{(1 + \frac{i}{100})^{\frac{du_2}{252}}} + ... + \frac{cp + VF}{(1 + \frac{i}{100})^{\frac{du_N}{252}}}$$

$$ cp =VF*[(1 + tx_{cp}/100)^{1/2} - 1] $$

$tx_{cp} = 10 $% a.a.

$ VF = 1000$ R$

In [1]:
import numpy as np

# interesting link https://www.tesourodireto.com.br/conheca/regras.htm
class titulo_pre_cupom(object):
    def __init__(self):
        self.vf = 1_000 # face value, the amount that will be received at maturity for each security purchased
        self.tx_c = 10

    def get_pv(self, list_du, tx):
        list_num = len(list_du)*[self.vf*((1 + self.tx_c/100)**(1/2) - 1)]
        list_num[-1] += self.vf
        list_pv = []
        for du, num in zip(list_du, list_num):
            temp_pv = num/(1 + tx/100)**(du/252)
            list_pv.append(temp_pv)
        pv = np.sum(list_pv)
        return pv, list_pv
 
    def venda_antecipada(self, list_du, tx_compra, tx_venda, du_venda, tipo='r'):
        list_num = len(list_du)*[self.vf*((1 + self.tx_c/100)**(1/2) - 1)]
        list_num[-1] += self.vf
        list_pv = []
        list_cup = []
        pv_compra,_ = titulo_pre_cupom().get_pv(list_du, tx_compra)
        for du, num in zip(list_du, list_num):
            if du_venda < du:
                temp_pv = num/(1 + tx_venda/100)**((du - du_venda)/252)
                list_pv.append(temp_pv)
            else:
                list_cup.append(num*(1 + tx_venda/100)**((du_venda - du)/252) if tipo == 'r' else num)
        pv_venda = np.sum(list_pv)
        valor_bruto_resgate = np.sum(list_cup) + pv_venda
        rb = valor_bruto_resgate/pv_compra - 1
        rba = (1 + rb)**(252/du_venda) - 1 
        
        return pv_venda, rba*100

## **Example 1**

In [4]:
tx = 9.87
list_du = [64, 187, 317, 439, 570, 692, 820, 944, 1071, 1195, 1320, 1444, 1570, 1693, 1823, 1946, 2074, 2197, 2325, 2449, 2576]
tit = titulo_pre_cupom()
pv, lista = tit.get_pv(list_du, tx)

In [5]:
lista

[47.65588719394776,
 45.51595433789959,
 43.35859649880499,
 41.42709960671666,
 39.448806874482834,
 37.691479515495175,
 35.931817526710866,
 34.30552427004203,
 32.716156707821995,
 31.23540597767872,
 29.810537831040797,
 28.46129696349343,
 27.15283149440275,
 25.933564795765125,
 24.7043698877013,
 23.59504854413868,
 22.49349162516519,
 21.483447229599967,
 20.480472393803478,
 19.55351527223079,
 400.7013886731155]

# **Example 2**

In [6]:
tx = 12.98
list_du = [120, 248, 372, 499]
tit = titulo_pre_cupom()
pv, list_pv = tit.get_pv(list_du, tx)

print(pv)

953.7545412135244


In [11]:
np.sum(list_pv)

953.7545412135244

## **Example 3**

In [13]:
tx = 14.00
list_du = [122, 250, 374, 501, 625, 750, 874, 1000]
tit = titulo_pre_cupom()
pv, list_vp = tit.get_pv(list_du, tx)
print(pv)
print(list_vp)

889.3326021048624
[45.80884954085998, 42.85932563656385, 40.18320135163156, 37.61545288083409, 35.266754541603824, 33.047520629198104, 30.984042699971212, 623.5674548241998]


## **Example 4**

In [8]:
tx_compra = 12.98
tx_venda = 14.00
list_du = [120, 248, 372, 499]
du_venda = 248 # number of working days passed since the date of purchase
tit = titulo_pre_cupom()
pv_venda, rba = tit.venda_antecipada(list_du, tx_compra, tx_venda, du_venda, tipo='r')
print('sell', pv_venda, ', profitability', rba)

#pv_compra,_ = tit.get_pv(list_du, tx_compra)


sell 966.2474844862911 , profitability 12.100210026107083
